In [80]:
# [1] Get lowest local price
# Done # 1. Get name, address, open times, menu items, & prices for Village from Yelp API
# Done # 2. Get top-rated 5 restaurants in 2 km with similar menu items
# 3. DISPLAY menu items & prices for Village + each restaurant

# [2] Get busy times & bad weather
# Done # 4. Get Village busy times from GMaps API
# Done # 5. Get temperature & rain near Village
# Done # 6. DISPLAY both

# -[3] DISPLAY Village menu with predicted prices using ANY ML algo:
# >IF
# 7.1 Temp is < 45 deg. Fahrenheit (note API returns Kelvin, convert to F)
# 7.2 It will snow or get moderate or heavy rain
# 7.3 Village is busier than usual
# >THEN
# Price should be more than lowest local price
# >ELSE
# Price should be lowest local price

# # SUBMISSION
# 1. Github code link
# 2. How to run
# 3. Display outputs for each step on a webpage or console

In [81]:
!cp /content/drive/MyDrive/keys.py /content

In [82]:
import keys

In [83]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import json

In [84]:
import re

In [85]:
#location of village
latitude=40.76665374624137
longitude=-73.52362919511015

In [86]:
yelp_response1 = requests.get(f'https://api.yelp.com/v3/businesses/search?latitude={latitude}&longitude={longitude}&radius=2000&categories=indpak&categories=vegetarian&categories=Indian&categories=Vegetarian', headers={'Authorization': f'Bearer {keys.yelp_api}'})
yelp_data=json.loads(yelp_response1.text)
print(yelp_data['businesses'][0]['name'])
print(yelp_data['businesses'][0]['location']['address1'])
open_status=yelp_data['businesses'][0]['business_hours'][0]['is_open_now']
print(open_status)

Village the soul of india
11 West Marie St
False


In [87]:
# village_menu_url='https://villagesoulofindia.com/app/order/menu'
village_menu_url='https://www.yelp.com/menu/village-the-soul-of-india-hicksville'

In [88]:
village_menu_response=requests.get(village_menu_url)
village_soup=BeautifulSoup(village_menu_response.text,'html.parser')

In [89]:
# village_items1 = [i.get_text() for i in village_soup.find_all('h5')]
village_items1 = [i.get_text() for i in village_soup.find_all('h4')]
village_items2=[re.sub(r'\s+', ' ', i.strip()) for i in village_items1]
village_items = [i.lower() for i in village_items2]
print(village_items[1:5])
print(len(village_items))

['medu vada in sambar bowl', 'medu vada in rasam bowl', 'combo', 'combo in sambar bowl']
186


In [90]:
# village_prices1=[p.get_text() for p in village_soup.find_all('div', class_="text-muted", style="text-align: center;")]
village_prices1=[p.get_text() for p in village_soup.find_all('li', class_='menu-item-price-amount')]
village_prices2=[p.replace('$','').strip() for p in village_prices1]
village_prices=[float(price) for price in village_prices2]
print(village_prices[1:5])
print(len(village_prices))

[6.99, 6.99, 5.99, 6.99]
186


In [91]:
df_village = pd.DataFrame(
    {'village_items': village_items,
     'village_prices': village_prices
    })
df_village.drop_duplicates(subset="village_items", keep="first", inplace=True)
df_village

,village_items,village_prices
0,medu vada,6.00
1,medu vada in sambar bowl,6.99
2,medu vada in rasam bowl,6.99
3,combo,5.99
4,combo in sambar bowl,6.99
...,...,...
181,masala tea hot,2.95
182,herbal tea hot,2.95
183,shake,6.00
184,fresh lime soda,7.45


In [92]:
competitiors_name=[]
competitiors_rating=[]
for i in range(1,20,1):
    competitiors_name.append(yelp_data['businesses'][i]['name'])
    competitiors_rating.append(yelp_data['businesses'][i]['rating'])

In [93]:
competitiors=pd.DataFrame({
    'Name':competitiors_name,
    'Rating':competitiors_rating
    })
competitiors.sort_values(by='Rating',ascending=False).head()

,Name,Rating
10,Taste of Mumbai,4.6
3,Kunga Kitchen,4.6
0,Kathis & Kababs,4.5
1,Taste Of Chennai,4.4
5,Dosa World,4.3


In [94]:
mumbai_url='https://www.yelp.com/menu/taste-of-mumbai-hicksville-3'

In [95]:
mumbai_response=requests.get(mumbai_url)
mumbai_soup=BeautifulSoup(mumbai_response.text,'html.parser')

In [96]:
mumbai_items1=[i.get_text() for i in mumbai_soup.find_all('h4')]
mumbai_items2=[re.sub(r'\s+', ' ', i.strip()) for i in mumbai_items1]
mumbai_items=[i.lower() for i in mumbai_items2]
print(mumbai_items[0:5])
print(len(mumbai_items))

['saag paneer', 'paneer tikka masala', 'shahi paneer', 'malai kofta', 'navratan korma']
147


In [97]:
mumbai_prices1=[p.get_text() for p in mumbai_soup.find_all('li', class_='menu-item-price-amount')]
mumbai_prices2=[re.sub(r'\s+', ' ', p.strip()).replace('$','') for p in mumbai_prices1]
mumbai_prices=[float(p) for p in mumbai_prices2]
print(mumbai_prices[0:5])
print(len(mumbai_prices))

[16.99, 17.99, 17.99, 17.99, 17.99]
147


In [98]:
df_mumbai=pd.DataFrame({
    'mumbai_items':mumbai_items,
    'mumbai_prices':mumbai_prices
    })
df_mumbai.drop_duplicates(subset="mumbai_items", keep="first", inplace=True)
df_mumbai

,mumbai_items,mumbai_prices
0,saag paneer,16.99
1,paneer tikka masala,17.99
2,shahi paneer,17.99
3,malai kofta,17.99
4,navratan korma,17.99
...,...,...
142,soft drink,2.50
143,chikoo shake,4.99
144,chaas,3.99
145,coffee,1.99


In [99]:
kunga_url='https://www.yelp.com/menu/kunga-kitchen-hicksville'

In [100]:
kunga_response=requests.get(kunga_url)
kunga_soup=BeautifulSoup(kunga_response.text,'html.parser')

In [101]:
kunga_items1=[i.get_text() for i in kunga_soup.find_all('h4')]
kunga_items2=[re.sub(r'\s+', ' ', i.strip()) for i in kunga_items1]
kunga_items=[i.lower() for i in kunga_items2]
print(kunga_items[0:5])
print(len(kunga_items))

['chilli fish dry', 'chilli shrimp dry', 'veg samosa', 'veg spring roll', 'chicken spring roll']
112


In [102]:
kunga_prices1=[p.get_text() for p in kunga_soup.find_all('li', class_='menu-item-price-amount')]
kunga_prices2=[re.sub(r'\s+', ' ', p.strip()).replace('$','') for p in kunga_prices1]
kunga_prices=[float(p) for p in kunga_prices2]
print(kunga_prices[0:5])
print(len(kunga_prices))

[15.59, 15.59, 5.19, 7.27, 7.27]
112


In [103]:
df_kunga=pd.DataFrame({
    'kunga_items':kunga_items,
    'kunga_prices':kunga_prices
    })
df_kunga.drop_duplicates(subset="kunga_items", keep="first", inplace=True)
df_kunga

,kunga_items,kunga_prices
0,chilli fish dry,15.59
1,chilli shrimp dry,15.59
2,veg samosa,5.19
3,veg spring roll,7.27
4,chicken spring roll,7.27
...,...,...
107,veg pad thai noodle,14.55
108,chicken pad thai noodle,15.59
109,veg fried rice,14.55
110,chicken fried rice,15.59


In [104]:
kathis_url='https://www.yelp.com/menu/kathis-and-kababs-hicksville'

In [105]:
kathis_response=requests.get(kathis_url)
kathis_soup=BeautifulSoup(kathis_response.text,'html.parser')

In [106]:
kathis_items1=[i.get_text() for i in kathis_soup.find_all('h4')]
kathis_items2=[re.sub(r'\s+', ' ', i.strip()) for i in kathis_items1]
kathis_items3= [re.sub(r'^\d+\.\s+', '', i) for i in kathis_items2]
kathis_items=[i.lower() for i in kathis_items3]
print(kunga_items[0:5])
print(len(kunga_items))

['chilli fish dry', 'chilli shrimp dry', 'veg samosa', 'veg spring roll', 'chicken spring roll']
112


In [107]:
kathis_prices1=[p.get_text() for p in kathis_soup.find_all('li', class_='menu-item-price-amount')]
kathis_prices2=[re.sub(r'\s+', ' ', p.strip()).replace('$','') for p in kathis_prices1]
kathis_prices=[float(p) for p in kathis_prices2]
print(kathis_prices[0:5])
print(len(kathis_prices))

[11.95, 11.95, 11.95, 11.95, 11.95]
62


In [108]:
df_kathis=pd.DataFrame({
    'kathis_items':kathis_items,
    'kathis_prices':kathis_prices
    })
df_kathis.drop_duplicates(subset="kathis_items", keep="first", inplace=True)
df_kathis

,kathis_items,kathis_prices
0,achari aaloo kathi roll,11.95
1,paneer tikka kathi roll,11.95
2,mushroom kathi roll,11.95
3,malai paneer kathi roll,11.95
4,egg omelette kathi roll,11.95
...,...,...
57,chicken lollipop (6 pcs),14.95
58,chicken drumsticks (6 pcs),14.95
59,gulab jamun,5.95
60,rasmalai,5.95


In [109]:
chennai_menu_url = 'https://www.yelp.com/menu/taste-of-chennai-hicksville'

In [110]:
chennai_menu_response=requests.get(chennai_menu_url)
chennai_soup=BeautifulSoup(chennai_menu_response.text,'html.parser')

In [111]:
chennai_items1 = [i.get_text() for i in chennai_soup.find_all('h4')]
chennai_items2 = [re.sub(r'\s+', ' ', i.strip()) for i in chennai_items1]
chennai_items=[i.lower() for i in chennai_items2]
print(chennai_items[0:5])
print(len(chennai_items))

['sambar', 'rasam', 'chicken soup', 'punjabi samosa (2 pcs)', 'onion pakora']
139


In [112]:
chennai_prices1=[p.get_text() for p in chennai_soup.find_all('li', class_='menu-item-price-amount')]
chennai_prices2=[re.sub(r'\s+', ' ', p.strip()).replace('$','') for p in chennai_prices1]
chennai_prices=[float(p) for p in chennai_prices2]
print(chennai_prices[0:5])
print(len(chennai_prices))

[7.49, 7.49, 8.99, 6.99, 8.99]
139


In [113]:
df_chennai = pd.DataFrame(
    {'chennai_items': chennai_items,
     'chennai_prices': chennai_prices
    })
df_chennai.drop_duplicates(subset="chennai_items", keep="first", inplace=True)
df_chennai

,chennai_items,chennai_prices
0,sambar,7.49
1,rasam,7.49
2,chicken soup,8.99
3,punjabi samosa (2 pcs),6.99
4,onion pakora,8.99
...,...,...
134,rasmalai,4.99
135,badam halwa,4.99
136,halwa a la mode,5.99
137,kulfi,2.99


In [114]:
dosa_world_url='https://www.yelp.com/menu/dosa-world-hicksville-2'

In [115]:
dosa_world_response=requests.get(dosa_world_url)
dosa_world_soup=BeautifulSoup(dosa_world_response.text,'html.parser')

In [116]:
dosa_items1=[i.get_text() for i in dosa_world_soup.find_all('h4')]
dosa_items2=[re.sub(r'\s+', ' ', i.strip()) for i in dosa_items1]
dosa_items=[i.lower() for i in dosa_items2]
print(dosa_items[0:5])
print(len(dosa_items1))

['thums up', 'fanta', 'limca', 'mineral water', 'sparkling water']
145


In [117]:
dosa_prices1=[p.get_text() for p in dosa_world_soup.find_all('li', class_='menu-item-price-amount')]
dosa_prices2=[re.sub(r'\s+', ' ', p.strip()).replace('$','') for p in dosa_prices1]
dosa_prices=[float(p) for p in dosa_prices2]
print(dosa_prices[0:5])
print(len(dosa_prices))

[3.5, 3.5, 3.5, 2.49, 1.75]
145


In [118]:
df_dosa_world=pd.DataFrame({
    'dosa_items':dosa_items,
    'dosa_prices':dosa_prices
    })
df_dosa_world.drop_duplicates(subset="dosa_items", keep="first", inplace=True)
df_dosa_world

,dosa_items,dosa_prices
0,thums up,3.50
1,fanta,3.50
2,limca,3.50
3,mineral water,2.49
4,sparkling water,1.75
...,...,...
140,rasmalai,4.99
141,gajar halwa,4.99
142,badam halwa,5.99
143,halwa a la mode,5.99


In [119]:
pip install rapidfuzz

In [120]:
from rapidfuzz import fuzz, process

In [121]:
threshold1=90
threshold2=95
threshold3=98
threshold4=98

In [122]:
vil=[]
mum=[]
for i in df_village['village_items']:
  for j in df_mumbai['mumbai_items']:
    if fuzz.QRatio(i,j) > threshold1 or fuzz.token_sort_ratio(i,j) > threshold2:
    # or fuzz.token_set_ratio(i,j) > threshold3 or fuzz.WRatio(i,j) > threshold4:
    # if fuzz.WRatio(i,j) > threshold:
      vil.append(i)
      mum.append(j)
df_vil_mum=pd.DataFrame({
    'village_items':vil,
    'mumbai_items':mum
    })
df_vil_mum.drop_duplicates(inplace=True)

In [123]:
df_vil_mum=pd.merge(df_vil_mum, df_mumbai, on='mumbai_items', how='left')

In [124]:
df_vil_mum

,village_items,mumbai_items,mumbai_prices
0,spring rolls,spring roll,7.99
1,tomato soup,tomato soup,3.99
2,mulligatawny soup,mulligatawny soup,3.99
3,katchumbar salad,kachumber salad,2.99
4,garden salad,garden salad,3.99
5,samosa chat,samosa chaat,5.99
6,vadda pav,vada pav,4.99
7,pav bhaji,pav bhaji,6.99
8,chilli mushroom,chilli mushroom,9.99
9,gobi lasuni,lasuni gobi,8.99


In [125]:
vil=[]
kun=[]
for i in df_village['village_items']:
  for j in df_kunga['kunga_items']:
    if fuzz.QRatio(i,j) > threshold1 or fuzz.token_sort_ratio(i,j) > threshold2:
    # or fuzz.token_set_ratio(i,j) > threshold3 or fuzz.WRatio(i,j) > threshold4:
    # if fuzz.WRatio(i,j) > threshold:
      vil.append(i)
      kun.append(j)
df_vil_kun=pd.DataFrame({
    'village_items':vil,
    'kunga_items':kun
    })
df_vil_kun.drop_duplicates(inplace=True)

In [126]:
df_vil_kun=pd.merge(df_vil_kun, df_kunga, on='kunga_items', how='left')

In [127]:
df_vil_kun

,village_items,kunga_items,kunga_prices
0,chilly paneer dry,chilli paneer dry,10.39
1,aloo gobi,aloo gobi,14.55
2,chana masala,chana masala,14.55
3,palak paneer,palak paneer,15.59
4,gulab jamun,gulab jamun,3.64
5,rasmalai,rasmalai,3.64


In [128]:
vil=[]
kat=[]
for i in df_village['village_items']:
  for j in df_kathis['kathis_items']:
    if fuzz.QRatio(i,j) > threshold1 or fuzz.token_sort_ratio(i,j) > threshold2:
    # or fuzz.token_set_ratio(i,j) > threshold3 or fuzz.WRatio(i,j) > threshold4:
    # if fuzz.WRatio(i,j) > threshold:
      vil.append(i)
      kat.append(j)
df_vil_kat=pd.DataFrame({
    'village_items':vil,
    'kathis_items':kat
    })
df_vil_kat.drop_duplicates(inplace=True)

In [129]:
df_vil_kat=pd.merge(df_vil_kat, df_kathis, on='kathis_items', how='left')

In [130]:
df_vil_kat

,village_items,kathis_items,kathis_prices
0,gobi manchurian dry,gobhi manchurian (dry),12.95
1,saag paneer,saag paneer,11.95
2,gulab jamun,gulab jamun,5.95
3,rasmalai,rasmalai,5.95
4,kheer,kheer,5.95
5,soda,soda,2.95


In [131]:
vil=[]
che=[]
for i in df_village['village_items']:
  for j in df_chennai['chennai_items']:
    if fuzz.QRatio(i,j) > threshold1 or fuzz.token_sort_ratio(i,j) > threshold2:
    # or fuzz.token_set_ratio(i,j) > threshold3 or fuzz.WRatio(i,j) > threshold4:
    # if fuzz.WRatio(i,j) > threshold:
      vil.append(i)
      che.append(j)
df_vil_che=pd.DataFrame({
    'village_items':vil,
    'chennai_items':che
    })
df_vil_che.drop_duplicates(inplace=True)

In [132]:
df_vil_che=pd.merge(df_vil_che, df_chennai, on='chennai_items', how='left')

In [133]:
df_vil_che

,village_items,chennai_items,chennai_prices
0,rasam,rasam,7.49
1,sambar,sambar,7.49
2,garden salad,garden salad,7.99
3,samosa chat,samosa chaat,9.99
4,gobi lasuni,lasuni gobi,11.49
5,set dosa,set dosa,9.99
6,masala dosa,masala dosa,10.99
7,chocolate dosa,chocolate dosa,8.99
8,palak paneer,palak paneer,15.49
9,paneer tikka masala,paneer tikka masala,15.49


In [134]:
vil=[]
dos=[]
for i in df_village['village_items']:
  for j in df_dosa_world['dosa_items']:
    if fuzz.QRatio(i,j) > threshold1 or fuzz.token_sort_ratio(i,j) > threshold2:
    # or fuzz.token_set_ratio(i,j) > threshold3 or fuzz.WRatio(i,j) > threshold4:
    # if fuzz.WRatio(i,j) > threshold:
      vil.append(i)
      dos.append(j)
df_vil_dos=pd.DataFrame({
    'village_items':vil,
    'dosa_items':dos
    })
df_vil_dos.drop_duplicates(inplace=True)

In [135]:
df_vil_dos=pd.merge(df_vil_dos, df_dosa_world, on='dosa_items', how='left')

In [136]:
df_vil_dos

,village_items,dosa_items,dosa_prices
0,medu vada,medu vada,5.99
1,dahi vada,dahi vada,6.99
2,batata vada,batata vada,5.99
3,samosa,samosa,5.99
4,tomato soup,tomato soup,4.99
5,rasam,rasam,4.99
6,sambar,sambar,1.75
7,mulligatawny soup,mulligatawny soup,4.99
8,katchumbar salad,katchumber salad,4.75
9,garden salad,garden salad,4.75


In [137]:
df_village_merge=pd.merge(df_village, df_vil_mum, on='village_items', how='left')
df_village_merge=pd.merge(df_village_merge, df_vil_kun, on='village_items', how='left')
df_village_merge=pd.merge(df_village_merge, df_vil_kat, on='village_items', how='left')
df_village_merge=pd.merge(df_village_merge, df_vil_che, on='village_items', how='left')
df_village_merge=pd.merge(df_village_merge, df_vil_dos, on='village_items', how='left')

In [138]:
df_village_merge.drop_duplicates(subset="village_items", keep="first", inplace=True)
df_village_merge

,village_items,village_prices,mumbai_items,mumbai_prices,kunga_items,kunga_prices,kathis_items,kathis_prices,chennai_items,chennai_prices,dosa_items,dosa_prices
0,medu vada,6.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,medu vada,5.99
1,medu vada in sambar bowl,6.99,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,medu vada in rasam bowl,6.99,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,combo,5.99,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,combo in sambar bowl,6.99,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
181,masala tea hot,2.95,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
182,herbal tea hot,2.95,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
183,shake,6.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
184,fresh lime soda,7.45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [139]:
print(df_village.shape)
print(df_vil_mum.shape)
print(df_vil_kun.shape)
print(df_vil_kat.shape)
print(df_vil_che.shape)
print(df_vil_dos.shape)
print(df_village_merge.shape)

(184, 2)
(25, 3)
(6, 3)
(6, 3)
(18, 3)
(44, 3)
(184, 12)


In [140]:
# df_village_merge.drop_duplicates(subset="village_items", keep="first", inplace=True)

In [141]:
df_village_merge.isna().sum()

,0
village_items,0
village_prices,0
mumbai_items,160
mumbai_prices,160
kunga_items,178
kunga_prices,178
kathis_items,178
kathis_prices,178
chennai_items,166
chennai_prices,166


In [142]:
# pip install fuzzywuzzy

In [143]:
# from fuzzywuzzy import fuzz, process
# from tabulate import tabulate

# # Replace these with your actual DataFrames

# all_lists = [df_village, df_mumbai, df_kunga, df_kathis, df_dosa_world]

# # Initialize merged dictionary
# merged_dishes = {}

# for i, lst in enumerate(all_lists):
#     for index, row in lst.iterrows():
#         dish = row[lst.columns[0]]  # Access the dish column by index
#         price = row[lst.columns[1]]  # Access the price column by index

#         # Find the best match for the current dish
#         result = process.extractOne(dish, merged_dishes.keys(), scorer=fuzz.token_sort_ratio)
#         if result:  # Check if a match is found
#             best_match = result[0]  # Extract only the match
#             score = result[1]       # Extract the similarity score
#             if score > 80:  # Set a threshold for similarity
#                 merged_dishes[best_match].append(price)
#             else:
#                 merged_dishes[dish] = [price]
#         else:
#             # No matches found, add dish to merged_dishes
#             merged_dishes[dish] = [price]

# # Calculate the average price for each dish
# final_dishes = {dish: round(sum(prices) / len(prices), 2) for dish, prices in merged_dishes.items()}
# final_df = pd.DataFrame(list(final_dishes.items()), columns=["Dish", "Average Price"])

# print(tabulate(final_df, headers="keys", tablefmt="grid"))


In [144]:
# with pd.ExcelWriter('menu.xlsx') as writer:
#     df_village.to_excel(writer, sheet_name='Village', index=False)
#     df_mumbai.to_excel(writer, sheet_name='Mumbai', index=False)
#     df_kunga.to_excel(writer, sheet_name='Kunga', index=False)
#     df_kathis.to_excel(writer, sheet_name='Kathis', index=False)
#     df_chennai.to_excel(writer, sheet_name='Chennai', index=False)
#     df_dosa_world.to_excel(writer, sheet_name='Dosa World', index=False)
#     df_village_merge.to_excel(writer, sheet_name='Village Merge', index=False)

In [145]:
!pip install pytz

In [146]:
import datetime
import pytz

In [147]:
# Get the current hour
timezone_str='America/New_York'
timezone = pytz.timezone(timezone_str)
current_time = datetime.datetime.now(timezone)
current_hour = current_time.hour

# Get the current date
today = datetime.date.today()
# Get the day of the week as an integer (0 = Monday, 6 = Sunday)
day_of_week = today.weekday()

print(day_of_week)
print(current_hour)

6
9


In [148]:
pip install --upgrade git+https://github.com/m-wrzr/populartimes

  Cloning https://github.com/m-wrzr/populartimes to /tmp/pip-req-build-jzb596kq
  Running command git clone --filter=blob:none --quiet https://github.com/m-wrzr/populartimes /tmp/pip-req-build-jzb596kq
  Resolved https://github.com/m-wrzr/populartimes to commit 989f66b63dd1b6a144c5ad4a7c3a200f7f8649ee
  Preparing metadata (setup.py) ... done


In [149]:
import populartimes

In [150]:
popular_times_data=populartimes.get_id(f'{keys.gmaps_api}', "ChIJPYSDLXWBwokRHLcHIl02Kh8")['populartimes'][day_of_week]['data'][current_hour]
print(popular_times_data)

0


In [151]:
weather_url=f'https://api.openweathermap.org/data/2.5/weather?lat={latitude}&lon={longitude}&appid={keys.openweather_api}'

In [152]:
weather_response=requests.get(weather_url)

In [153]:
weather_data=json.loads(weather_response.text)

In [154]:
temperature_k=weather_data['main']['temp']
temperature_f=round((temperature_k-273.15)*9/5+32,2)
print(temperature_f)
weather_main=str(weather_data['weather'][0]['main'])
print(weather_main)

32.81
Clear


In [155]:
if open_status == True and temperature_f < 45 or weather_main == "Snow" or weather_main == "Rain" or popular_times_data > 45:
    switch=1
    # Price should be more than lowest local price
else:
    switch=0
    # Price should be lowest local price
print(switch)

0


In [161]:
if switch==1:
  df_final=pd.DataFrame({
      'items':df_village_merge['village_items'],
      'prices':df_village_merge['village_prices']*1.1,
      'original prices':df_village['village_prices']
  })
else:
  df_final=pd.DataFrame({
      'items':df_village_merge['village_items'],
      'prices':df_village_merge[['village_prices','mumbai_prices','kunga_prices','kathis_prices','chennai_prices','dosa_prices']].min(axis=1),
      'original prices':df_village['village_prices']
  })

In [162]:
df_final

,items,prices,original prices
0,medu vada,5.99,6.00
1,medu vada in sambar bowl,6.99,6.99
2,medu vada in rasam bowl,6.99,6.99
3,combo,5.99,5.99
4,combo in sambar bowl,6.99,6.99
...,...,...,...
181,masala tea hot,2.95,2.95
182,herbal tea hot,2.95,2.95
183,shake,6.00,6.00
184,fresh lime soda,7.45,7.45
